In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch

import pyro
import pyro.distributions as dist
from pyro.contrib.autoguide import AutoDiagonalNormal
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import ClippedAdam

from src.utils import load_data

### Load data

In [2]:
X, y, k = load_data()
print('X-shape: {}, y-shape: {}'.format(X.shape, y.shape)) 
X.head()

X-shape: (2095, 35), y-shape: (2095,)


,youth_pop_5to18,commute_priv_vehicle,med_hhincome,avg_ann_daily_traffic,fragment_index,TotalPop,Men,Women,Hispanic,White,...,Transit,Walk,OtherTransp,WorkAtHome,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,894,0.634634,74837,8682.0,-3.061385,5403,2659,2744,75.8,2.3,...,38.6,2.9,0.0,0.0,2308,80.8,16.2,2.9,0.0,7.7
1,1158,0.494977,77991,16917.0,-2.915361,5915,2896,3019,62.7,3.6,...,44.6,1.4,0.5,2.1,2675,71.7,25.3,2.5,0.6,9.5
2,1120,0.422405,32354,22712.0,-0.227456,5879,2558,3321,65.1,1.6,...,45.5,8.6,1.6,1.7,2120,75.0,21.3,3.8,0.0,8.7
3,281,0.199795,34635,124767.0,3.029461,2591,1206,1385,55.4,9.0,...,63.9,3.0,2.4,6.2,1083,76.8,15.5,7.7,0.0,19.2
4,1998,0.286795,23423,10219.0,0.165237,8516,3301,5215,61.1,1.6,...,68.2,4.3,1.0,0.0,2508,71.0,21.3,7.7,0.0,17.2


##### Prepare data for model.

In [3]:
# standardize input features
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std

In [6]:
# Prepare data for Pyro model
X = torch.tensor(np.array(X)).float()
y = torch.tensor(y).float()
k = torch.tensor(k).long()
n_k = len(k.unique())

<ipython-input-6-d36d7a574743>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y).float()
<ipython-input-6-d36d7a574743>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  k = torch.tensor(k).long()


### Define model

**Generative Process**
<ol>
    <li> Draw parameter $r \sim \text{Gamma}(\alpha, \beta)$ . </li>
    <li> Draw parameter $\sigma \sim \text{HalfCauchy}(1)$. </li>
    <li> For each group $k \in \{1, \dots, K\}$ </li>
    <ol>
        <li> Draw linear coefficients $\beta_k \sim N(\beta|\textbf{0},\lambda\textbf{I})$</li>
    </ol>
    <li> For the i'th observation </li>
    <ol>
        <li> Draw $\psi_n \sim N(\psi_n | \beta_{k_n}^T\textbf{X}'_n+f_{\text{nnet}}(\textbf{X}^*_n), \sigma^2)$</li>
        <li> Compute $p_n$. </li>
        <li> Draw $y_n \sim \text{NB}(r, p_n)$</li>
    </ol>
</ol>

In [7]:
# Set model hyper-parameters.
alpha_r = 0.5
beta_r = 1
beta_mu = 0
beta_sigma = 1

In [8]:
(N, n_cat) = X.shape

alpha_mu = pyro.sample("alpha_mu", dist.Normal(torch.zeros(n_cat), 
                                                   10.*torch.ones(n_cat))) # Prior for the bias mean
alpha_sigma  = pyro.sample("alpha_sigma",  dist.HalfCauchy(10.*torch.ones(n_cat))) # Prior for the bias standard deviation
beta  = pyro.sample("beta", dist.Normal(torch.zeros(N, n_cat), 
                                        10.*torch.ones(N, n_cat))) # Priors for the regression coefficents

In [18]:
def hierarchical_model(X, k, n_k, obs=None):
    (N, n_cat) = X.shape
    
    # Draw parameters.
    r = pyro.sample("r", dist.Gamma(alpha_r*torch.ones(N, n_cat), beta_r*torch.ones(N, n_cat)).to_event())
    sigma = pyro.sample("sigma", dist.HalfCauchy(torch.ones(N, n_cat)).to_event())
    
    with pyro.plate("group", n_k):
        # Draw the parameters for each group
        beta = pyro.sample(
            "beta", 
            dist.Normal(beta_mu*torch.ones(n_cat), beta_sigma*torch.ones(n_cat)).to_event()
        )
        
    with pyro.plate("data", N):
        psi_mu = beta[k] * X
        print(sigma.shape)
        print(psi_mu.shape)
        psi = pyro.sample("psi", dist.Normal(psi_mu, sigma))
        pred = pyro.sample(
            "pred",
            dist.NegativeBinomial(r, logits=psi),
            obs=obs
        )

    return pred

# Use this helper to check our models are correct.
def test_model(model, guide, loss):
    pyro.clear_param_store()
    loss.loss(model, guide)
    
test_model(hierarchical_model(X, k, n_k), hierarchical_model, Trace_ELBO())

torch.Size([2095, 35])
torch.Size([2095, 35])


ValueError: Shape mismatch inside plate('data') at site psi dim -1, 2095 vs 35

In [19]:
%%time

# Define guide function.
guide = AutoDiagonalNormal(hierarchical_model)

# Reset parameter values.
pyro.clear_param_store()

# Define the number of optimization steps.
n_steps = 1000

# Setup the optimizer.
adam_params = {"lr": 0.01}
optimizer = ClippedAdam(adam_params)

# Setup the inference algorithm.
elbo = Trace_ELBO(num_particles=3)
svi = SVI(hierarchical_model, guide, optimizer, loss=elbo)

# Do gradient steps.
for step in range(n_steps):
    elbo = svi.step(X, k, n_k, y)
    if step % 500 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

torch.Size([2095, 35])
torch.Size([2095, 35])


ValueError: Shape mismatch inside plate('data') at site psi dim -1, 2095 vs 35
Trace Shapes:            
 Param Sites:            
Sample Sites:            
       r dist   | 2095 35
        value   | 2095 35
   sigma dist   | 2095 35
        value   | 2095 35
    beta dist 5 |   35   
        value 5 |   35   
Trace Shapes:
 Param Sites:
Sample Sites: